In [7]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from io import StringIO 

import io as io
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras

In [2]:
#Postgres credentials
pw = 'smedatabase'
db = "postgres" 
user='postgres'
host='smedatabase.cjxtg8rwmpmr.eu-west-1.rds.amazonaws.com'
port = '5432'


#s3 credentials
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

bucket_name =  "sme-dataset-bucket"
Prefix='orders_data/'
res = s3.list_objects_v2(
    Bucket=bucket_name,
    Prefix=Prefix)

In [4]:
def create_server_connection():
    connection = None
    try:

        connection = psycopg2.connect(
            database=db,
            user=user,
            password=pw,
            host=host,
            port=port
        )
        
        print("PostgreSql Database connection successful")
    except:
        print("Error - Coonection to database failed")

    return connection


connection = create_server_connection()

PostgreSql Database connection successful


In [18]:
def read_and_convert_excel_to_dataframe(df_name,file_path):

    obj = s3.get_object(Bucket= bucket_name, Key= Prefix+file_path)
    data = obj['Body'].read()
    df_name = pd.read_excel(io.BytesIO(data), skiprows = range(0,14))
    
    #Setting the header
    df_name = pd.DataFrame(df_name)
    new_header = df_name.iloc[0] 
    df_name = df_name[1:] 
    df_name.columns = new_header
    
    #formatting the header
    for column in df_name.columns:
        new = '_'.join(column.split()).lower()
        df_name.rename(columns = {column:new}, inplace = True)
        #df_name['client_name'] = df_name.client_name.to_title()
    
    return df_name 


read_and_convert_excel_to_dataframe('Sample Dataset1','Sample Dataset1.xlsx') 

,sno,client_name,firm_owner_name,gender,dob,county,sub_county,ward,fi_branch,group_name,...,disb_amount_iv,disb_date_iv,column2,disb_date_v,commitment_kes,leverage_value_kes,source,other_source,column1,job_type_hrs
1,212,Business 1,Name 1,Male,1967-01-26 00:00:00,Samburu,Sub-county a,Central,Marsabit,NaN,...,NaN,NaN,NaN,NaN,9000000,4000000,Revolving Portfolio,NaN,NaN,NaN
2,41,Business 2,Name 2,Male,1976-01-01 00:00:00,Garissa,Sub-county b,Central,Marsabit,NaN,...,NaN,NaN,NaN,NaN,4000000,12000000,Revolving Portfolio,NaN,NaN,NaN
3,34,Business 3,Name 3,Female,1963-11-25 00:00:00,Marsabit,Lamu,Lamu,Kilifi,NaN,...,NaN,NaN,NaN,NaN,4000000,75000000,Revolving Portfolio,NaN,NaN,NaN
4,35,Business 4,Name 4,Female,1963-11-25 00:00:00,Isiolo,Garsen,Garsen,Kilifi,NaN,...,NaN,NaN,NaN,NaN,30000000,75000000,Revolving Portfolio,NaN,NaN,NaN
5,57,Business 5,Name 5,Male,1966-02-06 00:00:00,Lamu,NaN,Lamu West,Mpeketoni,NaN,...,NaN,NaN,NaN,NaN,200000,600000,Revolving Portfolio,NaN,NaN,NaN


In [20]:
def load_df_to_postgres(conn, df, table):

    tuples = [list(row) for row in df.itertuples(index=False)]


    cols = ','.join(list(df.columns))
    
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)                   
    cursor = conn.cursor()
    
    try:
        extras.execute_values(cursor, query, tuples)
        #cursor.execute(query)
        conn.commit()
    
    except (Exception, psycopg2.DatabaseError):
        print("Error: %s")
        conn.rollback()
        cursor.close()
        return 1
    except psycopg2.InterfaceError:
        pass
    print("load to postgresql successful")
    cursor.close()

    
    
    
    
load_df_to_postgres(connection, 'Sample Dataset1', 'smedata')

AttributeError: 'str' object has no attribute 'itertuples'